In [32]:
import re
import os.path
import uritools
import numpy as np
import scipy as sp
import pandas as pd
from bs4 import BeautifulSoup

import seaborn as sns
import matplotlib.pyplot as plt

from pyspark.sql import *
import pyspark.sql.functions as f # wierd that I have to do that

In [2]:
# General parameters
%matplotlib inline
plt.style.use('seaborn')#switch to seaborn style
plt.rcParams["figure.figsize"] = [16,10]

DATA_FOLDER = './dataCleaned/'

In [31]:
# Importing ingredients to Pandas DF
ingredient_df = pd.read_csv(DATA_FOLDER + 'cleaned_ing.csv', sep='\t',  header=None, encoding = "utf-8")
ingredient_df.columns = ['ID', 'Title', 'Ingredients']

# Importing nutrition to Pandas DF, values are in [mg]!
nutrition_df = pd.read_csv(DATA_FOLDER + 'cleaned_nutri.csv', sep='\t',  header=None, encoding = "utf-8")
nutrition_df.columns = ['ID', 'kcal', 'carb', 'fat', 'prot', 'sodium', 'chol']

# Recipe Pandas DF, drop nan and ? values
recipe_df = pd.merge(ingredient_df, nutrition_df, how='inner', on='ID').dropna()
recipe_df = recipe_df[~recipe_df['kcal'].isin(['?'])].reset_index(drop=True)
recipe_df.sample(5)

,ID,Title,Ingredients,kcal,carb,fat,prot,sodium,chol
27811,cd0b4a28626befaebfcce55647d94067,Kentucky Mint Julep Cake Recipe,3 cups cake flour|2 cups white sugar|1 teaspoo...,562.8,75.9,25.8,7.5,427.8,100.6
30424,f3f1d205e7ae0f8e07acab823476b255,Teriyaki Tilapia My Mock Unagi) Recipe,1 tablespoon oil|5 tilapia fillets|1/2 cup bro...,125.0,23.1,2.7,3.0,1617.4,0.0
29603,4bf3271c58451dc20f8e3a7b97ee3785,Cake In A Cup Mix - Cupcakes Recipe,1 (18 1/2 ounce) box cake mix|1 (3 1/2 ounce) ...,521.7,109.7,8.5,3.1,624.0,0.0
31280,a21b521bd90c958ffdafc5dcc0f2a1f1,Artichoke Pasta Casserole Recipe,1 (10 ounce) package pasta shells|2 (7 ounce) ...,505.6,63.3,21.2,16.1,435.0,39.0
33598,005494c871460c7f368c649fad3363de,Prime Rib Soup Recipe,"4 beef ribs, trimmed from a cooked prime rib r...",197,11.5,8.7,18,655,41
